파이썬 입문 실습용 파일을 종합하여 24.11 외대 대학원 특강용으로 재구성.

마이닝에 적합한 내용으로 구성함.

작성일: 24년 10-11월


**순서**

파이썬 사용 환경 만들기

텍스트 파일 준비하기(txt 또는 csv)

전처리하기

# 라이브러리, 모델, 토크나이저 로드

In [5]:
pip install transformers torch matplotlib

In [6]:
import torch
print(torch.cuda.is_available())  # GPU가 사용 가능한지 확인

False


In [11]:
import numpy as np
import pandas as pd
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
import torch
import re
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from transformers import BertTokenizer, BertForMaskedLM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# 독일어 BERT 모델 및 토크나이저 로드
# bert-base-german-cased 로 하거나 아니면 dbmdz/bert-base-german-cased
model_name = "dbmdz/bert-base-german-cased"
tokenizer = BertTokenizer.from_pretrained(model_name) #모델명 바뀌면 위에서 재정의만.
model = BertForMaskedLM.from_pretrained(model_name)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

# 코퍼스 로드 (또는 직접 입력)

In [ ]:
#코랩인 경우

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#로컬인 경우

filename = 'sample.txt'

# 파일 읽기
with open(filename, 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)

#파일이 같은 디렉터리에 있지 않다면 filename에 파일의 절대 경로 또는 상대 경로를 지정

# 전처리

## 독일어 전처리 (spacy)

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_name = "dbmdz/bert-base-german-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)


beispiel = "Das ist ein Beispieltext auf Deutsch."

tokens = tokenizer.tokenize(beispiel)
tokens_with_tags = nlp(beispiel)

print(tokens)

for token in tokens_with_tags:
    print(f"단어: {token['word']}, 품사 태그: {token['entity']}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


['Das', 'ist', 'ein', 'Beispiel', '##text', 'auf', 'Deutsch', '.']


KeyError: 'entity'

In [10]:
# 텍스트 전처리
    # text = text.lower() 소문자변환 필요시
    # text.replace(',', '') 는 구나 절이 있을수도 있으므로 하지 않음. 콜론은 상황에 맞게 설정.
def preprocess_text(text):
    text = text.replace('.', '').replace('!', '').replace('?', '').replace('"', '').replace("'", '')
    return text

    # 텍스트를 단어 단위로 분할(필요시)
def split_text_into_words(text):
    words = text.split()
    return words

In [ ]:
# 입력 문장
text = """
Die Handelsbilanz der koreanischen Industrie für medizinische Geräte erreichte einen Gewinn von 587,8 Milliarden Won und verzeichnete damit vier Jahre in Folge schwarze Zahlen."""

## 한국어 전처리 (Konlpy)

In [1]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 18.9 MB/s eta 0:00:00


In [3]:
from konlpy.tag import Kkma
import nltk
from nltk.tag import pos_tag
kkma = Kkma()
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

print(kkma.sentences(u'네, 안녕하세요. 반갑습니다.'))

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


['네, 안녕하세요.', '반갑습니다.']


# 빈도 수 체크하기

# 워드클라우드 만들기

# 임베딩

TF-IDF 문서유사도

# 감성분석